In [1]:
import warnings
warnings.filterwarnings("ignore")
from trainer import load_and_preprocess_data
from tool import utils as ul
from tool import config as cfg
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import numpy as np


2024-02-22 12:21:20.015456: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 12:21:20.015535: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 12:21:20.015555: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 12:21:20.021551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 12:21:21.337507: W tensorflow/compiler/

In [2]:
dataset = load_and_preprocess_data()
X = np.array(dataset.iloc[:, 3:])
y = dataset['label'].values

# Define the K-fold cross-validator
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

fold_no = 1

models_results = {'knn','lr','rf'}

fold_metrics = {
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    }


results_df = pd.DataFrame(columns=['Model', 'Fold', 'Accuracy', 'Precision', 'Recall', 'F1_Score'])
res = []

for train, test in kfold.split(X, y):

    print(f'Training for fold {fold_no} ...')

    # Split data into training and validation sets
    X_train, X_val = X[train], X[test]
    y_train, y_val = y[train], y[test]
    
    print(f'X_train: {X_train.shape}')
    print(f'X_val: {X_val.shape}')
    
    for model_name in models_results:
        
        predicted_classes = ul.run_baseline(X_train, y_train, X_val, y_val, type='multi',method=model_name)
        
        # Save metrics
        print(f'predicted_classes: {predicted_classes}')
        accuracy  = accuracy_score(y_val, predicted_classes)
        precision = precision_score(y_val, predicted_classes, average='macro', zero_division=0)
        recall    = recall_score(y_val, predicted_classes, average='macro', zero_division=0)
        f1        = f1_score(y_val, predicted_classes, average='macro', zero_division=0)

        # Save metrics to dataframe
        res.append({
            'Model': model_name,
            'Fold': fold_no,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })
    
    # Increment fold number
    fold_no += 1  
    
results_df = pd.DataFrame(res)
results_df.to_csv('output/cross_validation_results_ml.csv', index=False)
display(results_df)    

FileNotFoundError: [Errno 2] No such file or directory: './featurebank/feature_esm2.feather'